In [ ]:
# google drive connect
Copied_path = '/content/drive/MyDrive/Colab Notebooks/MyPaper/ReWOO' # Paste target directory here
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir(Copied_path)

Mounted at /content/drive


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 945.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
INFO: pip is looking at multiple versions of multipro

In [1]:
Copied_path = '/content/drive/MyDrive/Colab Notebooks/MyPaper/ReWOO' # Paste target directory here
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir(Copied_path)

#!pip install -U langsmith==0.0.87
#!pip install -U langchain-core==0.1.23 #33
#!pip install -U langchain-openai==0.0.8

!pip install -U langchain-openai
!pip install -U langchain-community

from langchain_openai import OpenAI
!pip install fsspec==2023.9.2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# run.py의 앞부분 설정 부분만 여기서 실행

import argparse
import os
import warnings
warnings.filterwarnings("ignore")

'''
parser = argparse.ArgumentParser(description='Eval Arguments.')
parser.add_argument('--method',
                    type=str,
                    choices=['direct', 'cot', 'react', 'rewoo'],
                    help='Paradigm to use')

parser.add_argument('--exemplar',
                    type=str,
                    help='Input exemplar')

parser.add_argument('--toolset',
                    nargs='+',
                    default=['Google', 'Wikipedia', 'WolframAlpha', 'Calculator', 'LLM'],
                    help='Tools available to ALMs.')

parser.add_argument('--base_lm',
                    type=str,
                    default='text-davinci-003',
                    help='Base language model to use. Can be text-davinci-003, gpt-3.5-turbo or directory to alpca-lora')

parser.add_argument('--planner_lm',
                    type=str,
                    help='Base LM for Planner. Default to base_lm')

parser.add_argument('--solver_lm',
                    type=str,
                    help='Base LM for Solver. Default to base_lm')

parser.add_argument('--print_trajectory',
                    action='store_true',
                    help='Print reasoning traces to stdout (Only for ALMs)')

parser.add_argument('--key_path',
                    type=str,
                    default='./keys/',
                    help='Path where you store your openai.key and serpapi.key. Default to ./key/')

args = parser.parse_args()
'''

args = argparse.Namespace(
  method='direct', #''direct', 'rewoo'
  exemplar='hotpot_qa',
  dataset='hotpot_qa', # choices=["hotpot_qa", "trivia_qa", "gsm8k", "physics_question","sports_understanding", "strategy_qa", "sotu_qa"]
  sample_size=50,
  toolset=['Google', 'Wikipedia', 'WolframAlpha', 'Calculator', 'LLM'],
  base_lm='gpt-3.5-turbo-1106', # "gpt-3.5-turbo-1106", "gpt-4o", "gpt-4o-mini"
  planner_lm='gpt-3.5-turbo-1106', # "gpt-3.5-turbo-1106", "gpt-4o", "gpt-4o-mini"
  solver_lm='gpt-3.5-turbo-1106', # "gpt-3.5-turbo-1106", "gpt-4o", "gpt-4o-mini"
  save_result=False, #'store_true',
  seed=2024,
  print_trajectory=True,
  key_path='./keys/'
)


with open(os.path.join(args.key_path, 'openai.key'), 'r') as f:
    os.environ["OPENAI_API_KEY"] = f.read().strip()
with open(os.path.join(args.key_path, 'serpapi.key'), 'r') as f:
    os.environ["SERPAPI_API_KEY"] = f.read().strip()



In [5]:
# run_eval.py

from algos.PWS import *
from algos.notool import IO, CoT
from algos.react import ReactBase, ReactExtraTool
from utils.DataLoader import DataLoader
from utils.Evaluator import Evaluator
from utils.util import *


def save_data(dataset, data, save_path):
    dataset["preds"] = data["preds"]
    dataset["em"] = data["em"]
    dataset["f1"] = data["f1"]
    dataset["acc"] = data["acc"]
    dataset["wall_time"] = data["wall_time"]
    dataset["total_tokens"] = data["total_tokens"]
    dataset["steps"] = data["steps"]
    dataset["tool_cost"] = data["tool_cost"]
    dataset["token_cost"] = data["token_cost"]
    dataset["total_cost"] = data["total_cost"]
    dataset.to_csv(save_path, index=False)
    return dataset



dataset = DataLoader(args.dataset, seed=args.seed).load(sample_size=args.sample_size)
if args.method == 'direct':
    method = IO(model_name=args.base_lm)
    eval = Evaluator(args.dataset, dataset, method)
elif args.method == 'cot':
    method = CoT(model_name=args.base_lm, fewshot=DEFAULT_EXEMPLARS_COT[args.dataset])
    eval = Evaluator(args.dataset, dataset, method)
elif args.method == 'react':
    if args.dataset in ['hotpot_qa', 'trivia_qa']:
        method = ReactBase(model_name=args.base_lm, fewshot=DEFAULT_EXEMPLARS_REACT[args.dataset], verbose=False)
    else:
        method = ReactExtraTool(model_name=args.base_lm, available_tools=args.toolset,
                                fewshot=DEFAULT_EXEMPLARS_REACT[args.dataset], verbose=False)
    eval = Evaluator(args.dataset, dataset, method)
elif args.method == 'rewoo':
    if args.planner_lm is None:
        args.planner_lm = args.base_lm
    if args.solver_lm is None:
        args.solver_lm = args.base_lm
    method = PWS_Base(planner_model=args.planner_lm, solver_model=args.solver_lm,
                      fewshot=DEFAUL_EXEMPLARS_PWS[args.dataset], available_tools=args.toolset)
    eval = Evaluator(args.dataset, dataset, method)
else:
    raise NotImplementedError

responses, data = eval.run()
if args.save_result:
    save_data(dataset, data, f'./results/eval_{args.dataset}_{args.method}_{args.base_lm}.csv')
print(responses)


#if __name__ == '__main__':
#    main(args)

  0%|          | 0/3 [00:00<?, ?it/s]


******************* Start Evaluation *******************



  0%|          | 0/50 [00:00<?, ?it/s]

사용모델 :  gpt-3.5-turbo-1106 


 질문 Noble is a 2014 film written and directed by Stephen Bradley about the true life story of Christina Noble, a children's rights campaigner, charity worker and writer, it stars Sarah Greene, an Irish actress and singer, best known for portraying Helen McCormick in the West End and Broadway productions of what? 
 정답 The Cripple of Inishmaan 
 답변 The Cripple of Inishmaan. 
 판정 CORRECT

 질문 Noble is a 2014 film written and directed by Stephen Bradley about the true life story of Christina Noble, a children's rights campaigner, charity worker and writer, it stars Sarah Greene, an Irish actress and singer, best known for portraying Helen McCormick in the West End and Broadway productions of what? 
 정답 The Cripple of Inishmaan 
 답변 The Cripple of Inishmaan. 
 판정 CORRECT


  2%|▏         | 1/50 [00:02<01:47,  2.19s/it]

...


 질문 Who owns the news television channel who Andrew Ehinger is a reporter for? 
 정답 Altice USA 
 답변 News 12 Networks owns the news television channel Andrew Ehinger is a reporter for. 
 판정 INCORRECT
사용모델 :  gpt-3.5-turbo-1106 


 질문 What distinctive surface is the historic Merton Street, that runs parallel to High Street in the southeastern university city of Oxford, England? 
 정답 cobbled 
 답변 Cobbled surface. 
 판정 CORRECT

 질문 What distinctive surface is the historic Merton Street, that runs parallel to High Street in the southeastern university city of Oxford, England? 
 정답 cobbled 
 답변 Cobbled surface. 
 판정 CORRECT


 98%|█████████▊| 49/50 [01:41<00:02,  2.08s/it]


 질문 What distinctive surface is the historic Merton Street, that runs parallel to High Street in the southeastern university city of Oxford, England? 
 정답 cobbled 
 답변 Cobbled surface. 
 판정 CORRECT
사용모델 :  gpt-3.5-turbo-1106 


 질문 I'll Sail My Ship Alone has been recorded by which American country music singer? 
 정답 Mickey Leroy Gilley 
 답변 Patsy Cline 
 판정 INCORRECT

 질문 I'll Sail My Ship Alone has been recorded by which American country music singer? 
 정답 Mickey Leroy Gilley 
 답변 Patsy Cline 
 판정 INCORRECT


100%|██████████| 50/50 [01:43<00:00,  2.07s/it]


 질문 I'll Sail My Ship Alone has been recorded by which American country music singer? 
 정답 Mickey Leroy Gilley 
 답변 Patsy Cline 
 판정 INCORRECT
{'avg_em': 0.36, 'avg_f1': 0.42853968253968255, 'avg_acc': 0.62, 'avg_wall_time': 0.39336121559143067, 'avg_total_tokens': 47.58, 'avg_total_cost': 4.7579999999999995e-05, 'avg_steps': 1.0, 'avg_token_cost': 4.7579999999999995e-05, 'avg_tool_cost': 0.0}
